### Defining the Model Architecture and BackPropagation Method

In [3]:
import pandas as pd
import numpy as np

In [4]:
e = 1e1

# Activation Function
def sigmoid(x, grad = True):
    value = 1/(1 + np.exp(-1*x)) 
    if grad:
        # For Backpropagation
        grad = value*(1-value)
        return value, grad 
    return value

class Perceptron:
    def __init__(self, input_shape, output_shape, activation = sigmoid):
        self.weight = np.random.normal(size=(input_shape, output_shape)) # Random Initialization of Weights
        self.activation = activation
    
    def __call__(self, inp, grad):
        output = self.weight.T @ inp # Forward Pass
        if grad:
            output, grad = self.activation(output, grad) 
            return output, grad
        output = self.activation(output, grad) # Activation
        return output


class IrisClassifier:
    def __init__(self, input_shape, output_shape, num_hidden_layers, num_hidden_neurons):
        assert num_hidden_layers > 0, "There should be at least one hidden layer" # As required by the assignment
        
        if type(num_hidden_neurons) != list:
            num_hidden_neurons = [num_hidden_neurons] * num_hidden_layers
            
        assert num_hidden_layers == len(num_hidden_neurons), "The Number of Hidden Neurons should be in Number of Hidden Layers" # Required for forward pass
    
        # Making the Feed Forward Neural Network
        self.layers = [Perceptron(input_shape, num_hidden_neurons[0])] # First Layer
        for i in range(num_hidden_layers-1):
            self.layers.append(Perceptron(num_hidden_neurons[i], num_hidden_neurons[i+1])) # Hidden Layers
        self.layers.append(Perceptron(num_hidden_neurons[-1], output_shape)) # Output Layer
        self.grad = True
    
    # Entire Forward Pass
    def __call__(self, input):
        self.inputs = [input]
        self.gradients = []
        output, grad = self.layers[0](input, self.grad) # First Layer Forward Pass
        self.gradients.append(grad)
        for layer in self.layers[1:]:
            self.inputs.append(output)
            output, grad = layer(output, self.grad) # Passing the output of the previous layer to the next layer
            self.gradients.append(grad)
        return output
    
    # Backpropagation
    def update_weights(self, grad_loss):
        grad = self.gradients[-1] * grad_loss
        self.layers[-1].weight -= grad * np.expand_dims(self.inputs[-1], axis=-1)
        for i in range(len(self.layers)-1)[::-1]:
            grad = np.expand_dims(self.gradients[i], axis=-1) * self.layers[i+1].weight @ grad            
            self.layers[i].weight -= self.lr * np.expand_dims(self.inputs[i], axis=-1) @ np.expand_dims(grad, axis=-1).T 

    def train(self, X_train, y_train, X_test, y_test, learning_rate, n_epochs, logging_epochs = 10, validation_epochs = 100):
        validation_epochs = min(validation_epochs, n_epochs)
        self.lr = learning_rate
        for current_epoch in range(0, n_epochs + 1):
            total_loss = 0
            train_accuracy = 0
            for x, y in zip(X_train, y_train):
                pred = self(x) # Forward Pass
                loss, grad_loss = MSELoss(pred, y) # Loss Calculation
                total_loss += loss
                train_accuracy += (y == (pred>=0.5))
                self.update_weights(grad_loss) # Backpropagation
            if current_epoch % logging_epochs == 0:
                print("\nEpoch: ", current_epoch)
                print("Loss: ", total_loss/len(X_train))
                print("Train Accuracy: ", train_accuracy/len(X_train))
            
            # Validation
            if current_epoch % validation_epochs == 0:
                val_loss = 0
                val_accuracy = 0
                for x, y in zip(X_test, y_test):
                    pred = self(x)
                    loss, _ = MSELoss(pred, y)
                    val_loss += loss/len(X_test)
                    val_accuracy += (y == (pred>=0.5))
                print("Validation Loss: ", total_loss/len(X_test))
                print("Validation Accuracy: ", val_accuracy/len(X_test))
                
        print("\nTraining Finished!")
        print("Epoch: ", current_epoch)
        print("Loss: ", total_loss/len(X_train))
        print("Train Accuracy: ", train_accuracy/len(X_train))
        
        # Final Validation
        val_loss = 0
        val_accuracy = 0
        for x, y in zip(X_test, y_test):
            pred = self(x)
            loss, _ = MSELoss(pred, y)
            val_loss += loss/len(X_test)
            val_accuracy += (y == (pred>=0.5))
        print("Validation Loss: ", total_loss/len(X_test))
        print("Validation Accuracy: ", val_accuracy/len(X_test))

# Loss Function
def MSELoss(prediction, truth):
    loss = 0.5 * (prediction - truth) ** 2
    grad = prediction - truth
    return loss, grad

# Training the Model on Iris Dataset

In [25]:
train_data = pd.read_excel("iris_train.xlsx")
print(train_data.columns)

test_data = pd.read_excel("iris_test.xlsx")
print(test_data.columns)

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')
Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')


In [26]:
print(train_data.head())
print(train_data.tail())

print(test_data.head())
print(test_data.tail())

   sepal_length  sepal_width  petal_length  petal_width species
0           5.4          3.9           1.7          0.4  setosa
1           4.6          3.4           1.4          0.3  setosa
2           5.0          3.4           1.5          0.2  setosa
3           4.4          2.9           1.4          0.2  setosa
4           4.9          3.1           1.5          0.1  setosa
    sepal_length  sepal_width  petal_length  petal_width     species
75           5.7          3.0           4.2          1.2  versicolor
76           5.7          2.9           4.2          1.3  versicolor
77           6.2          2.9           4.3          1.3  versicolor
78           5.1          2.5           3.0          1.1  versicolor
79           5.7          2.8           4.1          1.3  versicolor
   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2         

In [28]:
print(train_data.species.unique())
print(test_data.species.unique())

['setosa' 'versicolor']
['setosa' 'versicolor']


In [30]:
def convert(x):
    values = ["setosa", "versicolor"]
    return values.index(x)

train_data["species"] = train_data["species"].apply(convert)
test_data["species"] = test_data["species"].apply(convert)

In [ ]:
print(train_data.head())
print(test_data.tail())

   sepal_length  sepal_width  petal_length  petal_width  species
0           5.4          3.9           1.7          0.4        0
1           4.6          3.4           1.4          0.3        0
2           5.0          3.4           1.5          0.2        0
3           4.4          2.9           1.4          0.2        0
4           4.9          3.1           1.5          0.1        0
    sepal_length  sepal_width  petal_length  petal_width  species
75           5.7          3.0           4.2          1.2        1
76           5.7          2.9           4.2          1.3        1
77           6.2          2.9           4.3          1.3        1
78           5.1          2.5           3.0          1.1        1
79           5.7          2.8           4.1          1.3        1


In [31]:
len(train_data), len(test_data)

(80, 20)

In [32]:
# Shuffle the Dataset
train_data = train_data.sample(frac = 1)

In [33]:
print(train_data.head())
print(train_data.tail())

    sepal_length  sepal_width  petal_length  petal_width  species
7            4.8          3.0           1.4          0.1        0
66           6.7          3.1           4.7          1.5        1
34           5.1          3.8           1.9          0.4        0
5            5.4          3.7           1.5          0.2        0
2            5.0          3.4           1.5          0.2        0
    sepal_length  sepal_width  petal_length  petal_width  species
22           5.2          3.5           1.5          0.2        0
74           5.6          2.7           4.2          1.3        1
21           5.0          3.4           1.6          0.4        0
9            5.8          4.0           1.2          0.2        0
12           5.1          3.5           1.4          0.3        0


In [34]:
np_train_data = train_data.to_numpy()
np_test_data = test_data.to_numpy()

In [35]:
X_train, y_train = np_train_data[:,:-1], np_train_data[:,-1]
X_test, y_test = np_test_data[:,:-1], np_test_data[:,-1]

In [ ]:
print(np_train_data.shape, np_test_data.shape)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((80, 5), (20, 5))

In [37]:
# IrisClassifier : input_shape, output_shape, num_hidden_layers, num_hidden_neurons
model = IrisClassifier(4, 1, num_hidden_layers = 1, num_hidden_neurons = [15])
# train method() : X_train, y_train, X_test, y_test, learning_rate, n_epochs, logging_epochs = 10, validation_epochs = 100
model.train(X_train, y_train, X_test, y_test, 0.0001 , 35 , 5, 10)


Epoch:  0
Loss:  [0.06153589]
Train Accuracy:  [0.875]
Validation Loss:  [0.24614358]
Validation Accuracy:  [1.]

Epoch:  5
Loss:  [0.00326697]
Train Accuracy:  [1.]

Epoch:  10
Loss:  [0.00164252]
Train Accuracy:  [1.]
Validation Loss:  [0.00657008]
Validation Accuracy:  [1.]

Epoch:  15
Loss:  [0.00109697]
Train Accuracy:  [1.]

Epoch:  20
Loss:  [0.00082412]
Train Accuracy:  [1.]
Validation Loss:  [0.00329646]
Validation Accuracy:  [1.]

Epoch:  25
Loss:  [0.00066044]
Train Accuracy:  [1.]

Epoch:  30
Loss:  [0.00055132]
Train Accuracy:  [1.]
Validation Loss:  [0.00220529]
Validation Accuracy:  [1.]

Epoch:  35
Loss:  [0.00047337]
Train Accuracy:  [1.]

Training Finished!
Epoch:  35
Loss:  [0.00047337]
Train Accuracy:  [1.]
Validation Loss:  [0.0018935]
Validation Accuracy:  [1.]
